In [7]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from utils import flat, trace
from tqdm import tqdm
import psutil
import torch
import gc
from itertools import product
from multiprocessing import Pool
from tqdm import tqdm
from data import RFCXDataset
from model import RFCXModel, kaggle_metric
from utils import sigmoid, CFG, frommelscale, tomelscale
import torch.nn.functional as F
import timm

In [9]:
a = torch.randn(3, 3, 3)
a

tensor([[[ 0.0851, -1.0511, -0.3357],
         [-0.4997, -0.5440,  1.8845],
         [-0.6018, -1.2933, -1.2714]],

        [[-0.6517,  0.7426, -1.3987],
         [-2.1782,  0.2993, -1.0012],
         [ 0.1995,  0.7417, -0.4769]],

        [[ 0.1139,  0.4296,  0.5730],
         [ 0.5436,  0.2419,  1.0395],
         [-0.2262,  0.6381, -1.4280]]])

In [13]:
torch.max(a, dim=(2,3))

TypeError: max() received an invalid combination of arguments - got (Tensor, dim=tuple), but expected one of:
 * (Tensor input)
 * (Tensor input, name dim, bool keepdim, *, tuple of Tensors out)
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, int dim, bool keepdim, *, tuple of Tensors out)


In [3]:
data_dir = '../../input/kaggle/train'
fnames = os.listdir(data_dir)
recording_ids = [x[:-5] for x in fnames]
print(len(fnames))
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
folds = joblib.load('folds.jl')
print(train_tp.shape)
display(train_tp.head())
print(train_fp.shape)
display(train_fp.head())

4729
(1216, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70


(7781, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [6]:
train_tp.groupby(level=0).size().value_counts()

1    1065
2      55
3       8
4       3
5       1
dtype: int64

In [10]:
def transform(x, offset):
    return frommelscale(tomelscale(x)+offset).astype(int)
offset = 100

df = pd.concat((train_tp, train_fp), axis=0)
for i in range(24):
    cur = df.loc[df['species_id']==i]
    tmp = transform(cur['f_min'], -offset).astype(str) + '-' + transform(cur['f_max'], offset).astype(str)
    print(i, tmp.unique())

0 ['5345-9080' '5345-8260']
1 ['3457-6211']
2 ['369-3343']
3 ['884-3138']
4 ['2085-4675' '2256-4470']
5 ['4315-11436' '4573-12358']
6 ['455-4880' '455-3548']
7 ['4230-12563']
8 ['3372-6109']
9 ['712-6314' '970-5802']
10 ['807-11924']
11 ['1595-6277']
12 ['455-3650']
13 ['26-986' '26-1601']
14 ['2256-6109' '3372-7133' '2514-4880']
15 ['26-1294']
16 ['2771-5084']
17 ['1280-5053' '1227-4367' '1141-4367' '1141-8158']
18 ['2857-5597']
19 ['197-3138']
20 ['1141-6724' '2085-6314']
21 ['2943-4572']
22 ['9720-15022']
23 ['6561-12395' '5852-12771']


In [15]:
train_df = pd.DataFrame(0, index=recording_ids, columns=range(24))
for i in range(24):
    cur = train_tp[train_tp['species_id']==i]
    train_df.loc[cur.index, i] = 1
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
73c482b64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
afaa8d38f,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
074cf066a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
de0c8a7f7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e568d45b5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# LOSS ERROR

In [7]:
data, logits, loss, grad_norm = joblib.load('error.jl')